In [1]:
from tqdm import tqdm
import pandas as pd
import os
import gzip
import zipfile
import shutil
from time import sleep
import pymorphy2
from pyaspeller import YandexSpeller
from lru import LRU
import pickle
from time import sleep
from concurrent.futures import ThreadPoolExecutor
import tarfile
import bz2
import sys
import numpy as np
with open('ill_url_ids.pickle', 'rb') as file:
    ill_url_ids = pickle.load(file)
def transform(arg, ill_url_ids):
    if arg not in ill_url_ids:
        return arg
    else:
        return ill_url_ids[arg]
alpha = np.array([0.41,0.16,0.105,0.08,0.06,0.05,0.35,0.3,0.25,0.2])
class URL_info():
    def __init__(self):
        self.total_shows = 0
        self.total_clicks = 0
        self.num_first_clicks = 0
        self.num_last_clicks = 0
        self.show_poses = np.zeros((10,),dtype='int')
        self.show_poses_cl = np.zeros((10,),dtype='int')
        self.click_poses = np.zeros((10,),dtype='int')
        self.time = 0
        self.clicks_on_higher_pos = 0
        self.clicks_on_lower_pos = 0
        self.clicks_after_this = 0
        self.clicks_before_this = 0
        self.is_click_on_higher_pos = 0
        self.is_click_on_lower_pos = 0

In [2]:
df_marks = pd.read_csv('train.marks.tsv', delimiter='\t', header=None)
df_marks = df_marks.rename(columns={0: "QueryId", 1: "DocumentId"})
df_marks = df_marks.drop(columns=[2])
df_example = pd.read_csv('sample.csv')
all_groups = df_marks.append(df_example)
all_groups = all_groups.reset_index()
all_groups = all_groups.drop(columns=['index'])
all_groups = all_groups.sort_values(by=['QueryId', 'DocumentId']).reset_index()
all_groups = all_groups.drop(columns=['index'])
rev_frame = all_groups.sort_values(by=['DocumentId', 'QueryId']).reset_index()
rev_frame = rev_frame.drop(columns=['index'])

In [3]:
vals = all_groups['DocumentId'].values

In [4]:
with open("doc_to_host.pickle", 'rb') as file:
    doc_to_host = pickle.load(file)
with open("url_simple.pickle", 'rb') as file:
    url_simple = pickle.load(file)
with open("host_simple.pickle", 'rb') as file:
    host_simple = pickle.load(file)

In [5]:
template = np.arange(1,11)
click_feats = dict()
click_feats1 = dict()

for i in range(2,11):
    click_feats['url_cumul_ctr{}'.format(i)] = []
for i in range(2,11):
    click_feats1['host_cumul_ctr{}'.format(i)] = []

click_feats['url_clicks_out'] = []
click_feats['url_shows_out'] = []
click_feats['url_clicks_out_prob'] = []
click_feats['url_shows_out_prob'] = []

click_feats1['host_clicks_out'] = []
click_feats1['host_shows_out'] = []
click_feats1['host_clicks_out_prob'] = []
click_feats1['host_shows_out_prob'] = []

click_feats['url_higher_prob'] = []
click_feats['url_lower_prob'] = []
click_feats['url_num_higher_cl'] = []
click_feats['url_num_lower_cl'] = []
click_feats1['host_higher_prob'] = []
click_feats1['host_lower_prob'] = []
click_feats1['host_num_higher_cl'] = []
click_feats1['host_num_lower_cl'] = []

In [6]:
for key in tqdm(vals):
    key1 = transform(key, ill_url_ids)
    
    for j in range(2,11):
        var = np.sum(url_simple[key1].click_poses[:j])/(np.sum(url_simple[key1].show_poses[:j])+0.0000001)
        click_feats['url_cumul_ctr{}'.format(j)].append(var)
        
    click_feats['url_clicks_out'].append(url_simple[key1].total_clicks - np.sum(url_simple[key1].show_poses_cl))
    click_feats['url_shows_out'].append(url_simple[key1].total_shows - np.sum(url_simple[key1].show_poses))
    var = url_simple[key1].total_clicks - np.sum(url_simple[key1].show_poses_cl)
    click_feats['url_clicks_out_prob'].append(var/(url_simple[key1].total_clicks+0.0000001))
    var = url_simple[key1].total_shows - np.sum(url_simple[key1].show_poses)
    click_feats['url_shows_out_prob'].append(var/(url_simple[key1].total_shows+0.0000001))
    
    click_feats['url_higher_prob'].append(url_simple[key1].is_click_on_higher_pos/(url_simple[key1].total_shows+0.0000001))
    click_feats['url_lower_prob'].append(url_simple[key1].is_click_on_lower_pos/(url_simple[key1].total_shows+0.0000001))
    click_feats['url_num_higher_cl'].append(url_simple[key1].is_click_on_higher_pos)
    click_feats['url_num_lower_cl'].append(url_simple[key1].is_click_on_lower_pos)

100%|██████████| 606050/606050 [01:41<00:00, 5964.99it/s]


In [7]:
url_simple = None

In [8]:
for key in tqdm(vals):
    key1 = transform(key, ill_url_ids)
    key1 = doc_to_host[key1]
    
    for j in range(2,11):
        var = np.sum(host_simple[key1].click_poses[:j])/(np.sum(host_simple[key1].show_poses[:j])+0.0000001)
        click_feats1['host_cumul_ctr{}'.format(j)].append(var)
        
    click_feats1['host_clicks_out'].append(host_simple[key1].total_clicks - np.sum(host_simple[key1].show_poses_cl))
    click_feats1['host_shows_out'].append(host_simple[key1].total_shows - np.sum(host_simple[key1].show_poses))
    var = host_simple[key1].total_clicks - np.sum(host_simple[key1].show_poses_cl)
    click_feats1['host_clicks_out_prob'].append(var/(host_simple[key1].total_clicks+0.0000001))
    var = host_simple[key1].total_shows - np.sum(host_simple[key1].show_poses)
    click_feats1['host_shows_out_prob'].append(var/(host_simple[key1].total_shows+0.0000001))
    
    click_feats1['host_higher_prob'].append(host_simple[key1].is_click_on_higher_pos/(host_simple[key1].total_shows+0.0000001))
    click_feats1['host_lower_prob'].append(host_simple[key1].is_click_on_lower_pos/(host_simple[key1].total_shows+0.0000001))
    click_feats1['host_num_higher_cl'].append(host_simple[key1].is_click_on_higher_pos)
    click_feats1['host_num_lower_cl'].append(host_simple[key1].is_click_on_lower_pos)

100%|██████████| 606050/606050 [01:35<00:00, 6315.55it/s]


In [9]:
with open("click_feats_extra.pickle", 'wb') as file:
    pickle.dump(click_feats, file)

In [10]:
with open("click_feats_extra1.pickle", 'wb') as file:
    pickle.dump(click_feats1, file)